In [1]:
"""Created on Monday Jan 6 11:03:00 
2019 Group 2
@authors: Sibel Yuksel"""

'Created on Monday Jan 6 11:03:00 \n2019 Group 2\n@authors: Sibel Yuksel'

In [2]:
!pip install textblob

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob

In [4]:
df = pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [11]:
url1 = 'https://omicron-formation.fr/3-gestion-et-organisation-de-la-cybersecurite/'
url2 = "https://omicron-formation.fr/formation-initiale-rgpd-intra-entreprise/"
url3 = "https://omicron-formation.fr/formations-rgpd/"

def get_Title(url : str) -> str :
    """
    This function extracts the title of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    html_titre = html_soup.title
    titre = html_titre.get_text()
    return titre

def get_Author(url : str) -> str :
    """
    This function extracts the name of the author of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    if html_soup.find("meta",{"name":"twitter:data1"}) is not None:
        author = html_soup.find("meta",{"name":"twitter:data1"})['content']
    else:
        author = 'no data'
    return author


def get_Time(url : str) -> str :
    """
    This function extracts the time of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    if html_soup.find("meta", {"property": "article:modified_time"}) is not None:
        date = html_soup.find("meta", {"property": "article:modified_time"})["content"]
    elif html_soup.find("meta", {"property": "article:published_time"}) is not None:
        date = html_soup.find("meta", {"property": "article:published_time"})["content"]
    else:
        return 'no data'
    return date[:10]

        
def get_lang(titre : str) -> str : 
    """
    This function analyse the webpage/article language
    """
    a = TextBlob(titre)
    langue = a.detect_language()
    return langue


def get_Img(url : str) -> str :
    """
    This function extracts the introducing picture of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    if html_soup.find("meta", {"property": "og:image"}) is not None:
        img = html_soup.find("meta", {"property": "og:image"})["content"]
    else:
        return 'no data'
    return img



def art_content_html(url : str) -> str :
    """
    This function extracts the html content of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    paragraphe = html_soup.find_all('p')
    content_html = " ".join([str(x) for x in paragraphe])
    return content_html
    
def art_content(url : str) -> str :
    """
    This function extracts the clean content of the webpage/article without 
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    paragraphe = html_soup.find_all('p')
    content = " ".join([x.text for x in paragraphe])
    return content

def get_Tag(url : str) -> str :
    """
    This function extracts different tags of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    if html_soup.find_all("a",{'class':"tag--link"}) is not None:
        html_tag = html_soup.find_all("a",{'class':"tag--link"})
        Tags = []
        for i in html_tag:
            tag_i = i.get_text()
            Tags.append(tag_i[1:])
        return Tags
    else:
        return "no data"



def remplir_dataFrame(url):
    """
    This function add different information of the webpage/article to the dataFrame
    """
    df.loc[0, 'src_type'] = 'xpath_source'
    df.loc[0, 'src_url'] = 'https://omicron-formation.fr/'
    df.loc[0, 'src_name'] = 'Omicron-formation'
    df.loc[0, 'art_url'] = url
    df.loc[0, 'art_auth'] = get_Author(url)
    df.loc[0, 'art_extract_datetime'] = get_Time(url)
    title = get_Title(url)
    df.loc[0, 'art_title'] = title
    df.loc[0, 'art_lang'] = get_lang(title)
    tag = get_Tag(url)
    if tag is None or tag==[]:
        tag = 'no data'
    df.loc[0, 'art_tag'] = tag
    df.loc[0, 'art_content_html'] = art_content_html(url)
    df.loc[0, 'art_content'] = art_content(url)
    df.loc[0, 'src_img'] = get_Img(url)
    return df

remplir_dataFrame(url3)


,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,09 88 44 33 24 Confidentialité prix d’un appel...,"<p>09 88 44 33 24</p> <p style=""text-align: ri...",no data,fr,Catalogue de formations au RGPD - Omicron Form...,https://omicron-formation.fr/formations-rgpd/,Omicron-formation,xpath_source,https://omicron-formation.fr/,https://omicron-formation.fr/wp-content/upload...,no data,no data
